Tách Band

In [9]:
import rasterio
import os

# Đường dẫn đến ảnh đa band
input_path = r"D:\Trí tuệ nhân tạo\Test\0226\rmvbg_0226.tif"
output_dir = r"D:\Trí tuệ nhân tạo\Test\0226\split_4bands"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

with rasterio.open(input_path) as src:
    for i in range(1, src.count + 1):  # src.count là số bands
        band = src.read(i)
        profile = src.profile
        profile.update(count=1)

        output_path = os.path.join(output_dir, f"band_{i}.tif")
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(band, 1)
        print(f"Saved {output_path}")

Saved D:\Trí tuệ nhân tạo\Test\0226\split_4bands\band_1.tif
Saved D:\Trí tuệ nhân tạo\Test\0226\split_4bands\band_2.tif
Saved D:\Trí tuệ nhân tạo\Test\0226\split_4bands\band_3.tif
Saved D:\Trí tuệ nhân tạo\Test\0226\split_4bands\band_4.tif


Tách features

In [10]:
import numpy as np
import rasterio
import os

# === Cấu hình ===
band_dir = r"D:\Trí tuệ nhân tạo\Test\0226\split_4bands"   # Thư mục chứa các band
output_dir = r"D:\Trí tuệ nhân tạo\sen_0226\S"           # Thư mục lưu các feature
os.makedirs(output_dir, exist_ok=True)

# === Đọc các band với đúng thứ tự phổ ===
with rasterio.open(os.path.join(band_dir, "band_1.tif")) as src:
    red = src.read(1).astype(np.float32)
    profile = src.profile

with rasterio.open(os.path.join(band_dir, "band_2.tif")) as src:
    green = src.read(1).astype(np.float32)

with rasterio.open(os.path.join(band_dir, "band_3.tif")) as src:
    blue = src.read(1).astype(np.float32)

with rasterio.open(os.path.join(band_dir, "band_4.tif")) as src:
    nir = src.read(1).astype(np.float32)

# === Tính các chỉ số đặc trưng phổ ===
eps = 1e-4  # Tránh chia cho 0

brightness = (red + green + blue) / 3
ndvi = (nir - red) / (nir + red + eps)
ndi_rb = (red - blue) / (red + blue + eps)
min_rgb = np.minimum.reduce([red, green, blue])
br_ratio = blue / (red + eps)

# === Lưu từng raster ===
features = {
    "brightness.tif": brightness,
    "ndvi.tif": ndvi,
    "ndi_rb.tif": ndi_rb,
    "min_rgb.tif": min_rgb,
    "br_ratio.tif": br_ratio,
}

profile.update(dtype=rasterio.float32, count=1)

for name, array in features.items():
    out_path = os.path.join(output_dir, name)
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(array, 1)
    print(f"✅ Saved {out_path}")


✅ Saved D:\Trí tuệ nhân tạo\sen_0226\S\brightness.tif
✅ Saved D:\Trí tuệ nhân tạo\sen_0226\S\ndvi.tif
✅ Saved D:\Trí tuệ nhân tạo\sen_0226\S\ndi_rb.tif
✅ Saved D:\Trí tuệ nhân tạo\sen_0226\S\min_rgb.tif
✅ Saved D:\Trí tuệ nhân tạo\sen_0226\S\br_ratio.tif
